This notebook demonstrates the result of the first round of data collection, collected in the San Francisco Bay Area by @shankari. The round had several shortcomings, some of which were addressed during the data collection and some of which were fixed before starting the second round of data collection.

## Import all the dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# For maps
import branca.element as bre

In [ ]:
# For displaying dates
import arrow

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sdmco1 = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "sfba_med_freq_calibration_only")

In [ ]:
import importlib
importlib.reload(ezev)

In [ ]:
pvmco1 = eipv.PhoneView(sdmco1)

### Issue #1: Multiple and missing transitions

While exploring the data after the collection was done, there were still inconsistencies in the transitions pushed to the server - there were a bunch of duplicate transitions, and two of the phones were missing start transitions for the second trip.

In [ ]:
# Commented out because this fails
# pvt3.validate()

In [ ]:
pvmco1.validate()

In [ ]:
evmco1 = eiev.EvaluationView()
evmco1.from_view_multiple_runs(pvmco1, "")

## Now for the results (calibration, phone view)!

### Battery drain over time (moving calibration)

Again, the moving calibration runs were not very useful in terms of battery drain, since there were too few points to be useful. We actually have more points on android, but we have almost nothing for the iOS medium accuracy runs.

Part of this is inherent in the definition of moving calibration, since it is unlikely that we will move for 10-15 hours at a time to collect the kind of data we have in the stationary case. And if our trip lasts for an hour, but we only read the battery level once an hour, we will end up with close to no data.

But with some native code changes, I think we can do better wrt at least recording the battery reading at the trip start and end.

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(12,12))

ezpv.plot_all_power_drain(ios_ax, pvmco1.map()["ios"], "calibration", "high")
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
ezpv.plot_all_power_drain(android_ax, pvmco1.map()["android"], "calibration", "high")
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(12,12))

ezpv.plot_all_power_drain(ios_ax, pvmco1.map()["ios"], "calibration", "medium")
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
ezpv.plot_all_power_drain(android_ax, pvmco1.map()["android"], "calibration", "medium")
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
(ifig, axes) = plt.subplots(ncols=0, nrows=0, figsize=(12,3))

ezpv.plot_separate_power_drain(ifig, pvmco1.map()["ios"], 4, "calibration", "stationary")
# android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
(ifig, axes) = plt.subplots(ncols=0, nrows=0, figsize=(12,3))

ezpv.plot_separate_power_drain(ifig, pvmco1.map()["android"], 4, "calibration", "stationary")
# android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
(ifig, axes) = plt.subplots(ncols=0, nrows=0, figsize=(16,4))

ezev.plot_separate_power_drain_multiple_runs(ifig, 2, evmco1.map("calibration")["ios"], "")
android_ax.legend(loc="upper center", bbox_to_anchor=(1, 0.5))
(ifig, axes) = plt.subplots(ncols=0, nrows=0, figsize=(16,4))

ezev.plot_separate_power_drain_multiple_runs(ifig, 2, evmco1.map("calibration")["android"], "")
android_ax.legend(loc="upper center", bbox_to_anchor=(1, 0.5))

In [ ]:
importlib.reload(ezev)

In [ ]:
def display_transitions_ranges_values(ax, phone_map, range_key, trip_id_pattern, offset_hr=1):
    for phone_label, phone_details in phone_map.items():
        curr_calibration_ranges = phone_details["{}_ranges".format(range_key)]
        sel_calibration_ranges = [cr for cr in curr_calibration_ranges if trip_id_pattern in cr["trip_id"]]
        for r in sel_calibration_ranges:
            battery_df = r["battery_df"]
            start_level = battery_df.battery_level_pct.iloc[0]
            battery_df["battery_change_from_start"] = battery_df.battery_level_pct - start_level
            battery_df["hr_new"] = battery_df.hr + offset_hr
            ret_axes = battery_df.plot(x="hr_new", y="battery_change_from_start", ax=ax, label=phone_label+"_"+r["trip_id"], ylim=(0, -100), sharey=True)

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(6,12))

display_transitions_ranges_values(ios_ax, pvmco1.map()["ios"], "calibration", "high_accuracy_stationary_0", 0)
display_transitions_ranges_values(ios_ax, pvmco1.map()["ios"], "calibration", "high_accuracy_stationary_1", 0)
display_transitions_ranges_values(ios_ax, pvmco1.map()["ios"], "calibration", "high_accuracy_stationary_2", 0)
# display_transitions_ranges_values(ios_ax, pvmco1.map()["ios"], "calibration", "medium_accuracy_stationary_0", 0)
# display_transitions_ranges_values(ios_ax, pvmco1.map()["ios"], "calibration", "medium_accuracy_stationary_1", 0)
# display_transitions_ranges_values(ios_ax, pvmco1.map()["ios"], "calibration", "medium_accuracy_stationary_2", 0)
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
display_transitions_ranges_values(android_ax, pvmco1.map()["android"], "calibration", "stationary", 0)
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))